In [ ]:
# Oh maybe we could just pick out each of the samples from each of them and use that...Then also extend to the other datasets.

# So yeah - let's feed ChatGPT and see what scripts it could generate for me

In [3]:
import json

# Step 1: Read both JSON files and load the data into memory
with open('../spider/dev.json') as f:
    spiderDataPoints = json.load(f)

with open('../sql_create_context_v4.json') as f:
    sqlCreateContextPoints = json.load(f)

In [4]:
print(len(spiderDataPoints))

1034


In [5]:
import sqlparse

def normalize_query(query):
    # Lowercase the query
    query = query.lower()
    
    # Parse and format the query to normalize the spacing
    parsed_query = sqlparse.parse(query)
    normalized_query = " ".join([sqlparse.format(str(stmt), strip_comments=True, strip_whitespace=True) for stmt in parsed_query])
    
    return normalized_query

# Test the function
query = "SELECT count(*) FROM pets WHERE weight  >  10"
normalized_query = normalize_query(query)
print(normalized_query)


select count(*) from pets where weight > 10


In [6]:
for createContextItem in sqlCreateContextPoints:
    if "answer" in createContextItem:
        # print(item1["answer"])
        createContextItem["normalised"] = normalize_query(createContextItem["answer"])

for spiderItem in spiderDataPoints:
    if "query" in spiderItem:
        # print(item2["query"])
        spiderItem["normalised"] = normalize_query(spiderItem["query"])

In [7]:
sqlCreateContextPoints

[{'question': 'How many heads of the departments are older than 56 ?',
  'context': 'CREATE TABLE head (age INTEGER)',
  'answer': 'SELECT COUNT(*) FROM head WHERE age > 56',
  'normalised': 'select count(*) from head where age > 56'},
 {'question': 'List the name, born state and age of the heads of departments ordered by age.',
  'context': 'CREATE TABLE head (name VARCHAR, born_state VARCHAR, age VARCHAR)',
  'answer': 'SELECT name, born_state, age FROM head ORDER BY age',
  'normalised': 'select name, born_state, age from head order by age'},
 {'question': 'List the creation year, name and budget of each department.',
  'context': 'CREATE TABLE department (creation VARCHAR, name VARCHAR, budget_in_billions VARCHAR)',
  'answer': 'SELECT creation, name, budget_in_billions FROM department',
  'normalised': 'select creation, name, budget_in_billions from department'},
 {'question': 'What are the maximum and minimum budget of the departments?',
  'context': 'CREATE TABLE department (bud

In [9]:


# Step 2 and 3: Iterate over each object in the second JSON file, 
# extract the "query" attribute, and find the matching object in the first JSON file
new_list = []
for spiderItem in spiderDataPoints:
    spiderItemNormalised = spiderItem.get("normalised")
    spiderQuestion = spiderItem.get("question")
    # print(query2)
    foundQuery2 = False
    for createContextItem in sqlCreateContextPoints:
        createContextItemNormalised = createContextItem.get("normalised")
        createContextQuestion = createContextItem.get("question")
        # print(answer1)
        if (spiderItemNormalised == createContextItemNormalised or spiderQuestion == createContextQuestion):
            createContextItem["db_id"] = spiderItem["db_id"]
            createContextItem["question"] = spiderItem["question"]
            createContextItem["answer"] = spiderItem["query"]

            print(createContextItem["db_id"])
            new_list.append(createContextItem)
            sqlCreateContextPoints.remove(createContextItem) 
            foundQuery2 = True
            break
    if not foundQuery2:
        print("Could not find a match for: " + spiderItemNormalised + " " + spiderQuestion)



concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
Could not find a match for: select name from stadium where stadium_id not in (select stadium_id from concert) What are the names of the stadiums without any concerts?
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
Could not find a match for: select major, age from stu

In [10]:
print(len(new_list))
print(len(spiderDataPoints))


936
1034


In [11]:
def remove_normalised_property(obj_list):
    for obj in obj_list:
        if "normalised" in obj:
            del obj["normalised"]
    return obj_list

In [12]:
result_list = remove_normalised_property(new_list)
print(result_list[0])


{'question': 'What is the count of singers?', 'context': 'CREATE TABLE singer (Id VARCHAR)', 'answer': 'SELECT count(*) FROM singer', 'db_id': 'singer'}


In [13]:
print(len(new_list))
print(len(spiderDataPoints))


936
1034


In [14]:
# Step 4: Save the new list to a new JSON file
with open('spider_create_context_val_db_id.json', 'w') as f:
    json.dump(result_list, f, indent=4)

In [64]:
from datasets import load_dataset
import json
import os

# Load the dataset from Hugging Face
dataset = load_dataset('lamini/spider_text_to_sql')

# Create a directory to store the JSON files
os.makedirs('lamini', exist_ok=True)

# Convert the dataset splits to lists of dictionaries
train_data = [dict(item) for item in dataset['train']]
validation_data = [dict(item) for item in dataset['validation']]

# Write the train split to a JSON file
with open('lamini/train.json', 'w') as train_file:
    json.dump(train_data, train_file, ensure_ascii=False, indent=4)

# Write the validation split to a JSON file
with open('lamini/validation.json', 'w') as validation_file:
    json.dump(validation_data, validation_file, ensure_ascii=False, indent=4)

print("Dataset has been downloaded and saved to 'lamini' directory.")


Dataset has been downloaded and saved to 'lamini' directory.
